<a href="https://colab.research.google.com/github/thayanneramires/Mercado_Trabalho_Amazonense_PNADC/blob/main/Mercado_Trabalho_Amazonense_Microdados_PNADC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de microdados da PNAD Contínua:
# Participação das mulheres amazonenses no mercado de trabalho

In [ ]:
# Instalando pacotes
install.packages("PNADcIBGE")
install.packages("dplyr")
install.packages("expss")
install.packages('maditr')
install.packages('openxlsx')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘minqa’, ‘numDeriv’, ‘mitools’, ‘RcppArmadillo’, ‘projmgr’, ‘RCurl’, ‘survey’, ‘timeDate’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘maditr’, ‘htmlTable’, ‘matrixStats’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Importando pacotes
library(PNADcIBGE)
library(dplyr)
library(maditr)
library(expss)
library(openxlsx)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



To aggregate data: take(mtcars, mean_mpg = mean(mpg), by = am)



Attaching package: ‘maditr’


The following objects are masked from ‘package:dplyr’:

    between, coalesce, first, last


The following object is masked from ‘package:base’:

    sort_by



Use 'expss_output_rnotebook()' to display tables inside R Notebooks.
 To return to the console output, use 'expss_output_default()'.



Attaching package: ‘expss’


The following objects are masked from ‘package:dplyr’:

    compute, contains, na_if, recode, vars, where




In [ ]:
# Importando os microdados
pnadc_2021 <- get_pnadc(year = 2021, quarter = 3,
            vars = c("Ano","Trimestre", "UF", "UPA", "Estrato", "V1028", "V2007", "V2009", "V2010", "VD3004",
            "VD4011","VD4009", "VD4001", "VD4002", "VD4003", "VD4004A", "VD4019","VD4020"),
            labels = TRUE, deflator = TRUE, design = FALSE)

# Indicadores de mercado de trabalho por gênero



In [ ]:
# Selecionando variáveis para o cálculo de indicadores de Mercado de Trabalho e filtrando por UF
indicadores_mt <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
tab_cells(V2007) %>%
tab_cols(VD4001, VD4002, VD4003,VD4004A) %>%
tab_weight(weight = V1028) %>%
tab_stat_cases(total_statistic = "w_cases",
total_label = "Total") %>%
tab_pivot()

# Renomeando variáveis
colnames(indicadores_mt) <-
c("genero","pnft", "pfft", "ocup","desocup", "ftp","fft","subocup")

# Construindo indicadores
indicadores_mt <-
as.data.frame(indicadores_mt %>%
mutate(pia = pnft+pfft, pea = pnft, foraft = pfft,  ocupadas = ocup, desocupadas = desocup, fnaftpotencial = ftp, foraftpotencial = fft, subocupadas = subocup,
tx_partp = ((ocup+desocup)/(pnft+pfft))*100, nocupacao = (ocup/(pnft+pfft))*100, tx_ocupacao = (ocup/(ocup+desocup))*100,
tx_desocup = (desocup/(ocup+desocup))* 100, ndesocup = (desocup/(pnft+pfft))*100, tx_comp_subut = ((desocup+subocup+ftp)/(ocup+desocup+ftp))* 100,
tx_comb_subocup = (desocup+subocup)/(ocup+desocup)* 100))

# Montando a tabela
indicadores_mt <-
drop_rc(as.data.frame(indicadores_mt) %>%
select(genero, pia, pea, foraft, ocupadas, desocupadas, fnaftpotencial, foraftpotencial, subocupadas, tx_partp, nocupacao, tx_ocupacao, tx_desocup, ndesocup, tx_comp_subut, tx_comb_subocup))
indicadores_mt

,genero,pia,pea,foraft,ocupadas,desocupadas,fnaftpotencial,foraftpotencial,subocupadas,tx_partp,nocupacao,tx_ocupacao,tx_desocup,ndesocup,tx_comp_subut,tx_comb_subocup
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,V2007|Homem,1537566,1144713.2,392853.1,1029535.6,115177.6,76920.35,315932.7,72324.08,74.44968,66.95878,89.93830,10.06170,7.490904,21.64496,16.37980
2,V2007|Mulher,1554094,820946.1,733148.1,671794.5,149151.6,128615.97,604532.2,79321.38,52.82473,43.22740,81.83174,18.16826,9.597332,37.60565,27.83045
3,V2007|#Total,3091660,1965659.3,1126001.2,1701330.1,264329.2,205536.32,920464.9,151645.46,63.57940,55.02965,86.55264,13.44736,8.549748,28.62529,21.16209


In [ ]:
# Exportando Resultado
write.xlsx(indicadores_mt, 'mt_2021.xlsx')

# Rendimento por gênero

In [ ]:
# Rendimento médio efetivo
rendamedia <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
group_by(V2007) %>%
summarise("Rendimento médio" =
weighted.mean(VD4020,
w = V1028,
na.rm = T))

rendamedia

V2007,Rendimento médio
<fct>,<dbl>
Homem,1796.030
Mulher,1602.918


In [ ]:
# Exportando Resultado
write.xlsx(rendamedia, 'rendimento_2021.xlsx')

# Participação no mercado de trabalho por gênero, segundo cor ou raça

In [ ]:
# Mercado de trabalho por cor/raça
indicadores_mt_cor <- pnadc_2021 %>%
  filter(UF == "Amazonas") %>%
  tab_cells(interaction(V2007, V2010)) %>%  # Concatenar gênero e cor/raça usando interaction
  tab_cols(VD4001, VD4002, VD4003, VD4004A) %>%
  tab_weight(weight = V1028) %>%
  tab_stat_cases(total_statistic = "w_cases", total_label = "Total") %>%
  tab_pivot()

# Ajustar os nomes das colunas
colnames(indicadores_mt_cor) <- c("cor", "pnft", "pfft", "ocup", "desocup", "ftp", "fft", "subocup")

# Calcular métricas
indicadores_mt_cor <- as.data.frame(indicadores_mt_cor %>%
  mutate(pia = pnft + pfft,
         pea = pnft,
         ocupadas = ocup,
         desocupadas = desocup,
         tx_partp = ((ocup + desocup) / (pnft + pfft)) * 100,
         tx_desocup = (desocup / (ocup + desocup)) * 100,
         tx_comp_subut = ((desocup + subocup + ftp) / (ocup + desocup + ftp)) * 100))

# Selecionar colunas de interesse
indicadores_mt_cor <- drop_rc(as.data.frame(indicadores_mt_cor) %>%
  select(cor, pia, pea, ocupadas, desocupadas, tx_partp, tx_desocup, tx_comp_subut))

# Exibir o resultado
indicadores_mt_cor


,cor,pia,pea,ocupadas,desocupadas,tx_partp,tx_desocup,tx_comp_subut
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,"interaction(V2007, V2010)|Homem.Branca",218581.462,162152.476,147843.109,14309.3667,74.18400,8.824637,18.46503
2,"interaction(V2007, V2010)|Mulher.Branca",254349.751,143473.475,116462.781,27010.6942,56.40795,18.826263,33.97835
3,"interaction(V2007, V2010)|Homem.Preta",62833.998,49049.178,45706.014,3343.1639,78.06153,6.815943,17.81270
4,"interaction(V2007, V2010)|Mulher.Preta",53817.590,31706.435,26157.976,5548.4583,58.91463,17.499471,35.70961
5,"interaction(V2007, V2010)|Homem.Amarela",5393.491,4763.286,4283.942,479.3438,88.31544,10.063301,NA
6,"interaction(V2007, V2010)|Mulher.Amarela",5186.086,3424.719,2558.424,866.2950,66.03668,25.295362,NA
7,"interaction(V2007, V2010)|Homem.Parda",1222088.967,908120.647,812506.058,95614.5892,74.30888,10.528842,22.26543
8,"interaction(V2007, V2010)|Mulher.Parda",1201231.297,628076.720,514708.716,113368.0042,52.28608,18.050025,37.62153
9,"interaction(V2007, V2010)|Homem.Indígena",28668.362,20627.638,19196.493,1431.1455,71.95262,6.938000,29.31842


In [ ]:
# Exportando Resultado
write.xlsx(indicadores_mt_cor, 'cor_2021.xlsx')

# Participação no mercado de trabalho por gênero, segundo grupos de idade

In [ ]:
pnadc_2021$faixa_idade <-
recode(pnadc_2021$V2009,
14:17 ~ "14 a 17 anos",
18:24 ~ "18 a 24 anos",
25:39 ~ "25 a 39 anos",
40:59 ~ "40 a 59 anos",
60:120 ~ "60 anos ou mais")

In [ ]:
# Percentual no mercado de trabalho por faixa de idade
participacao_idade <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
tab_cells(interaction(V2007, faixa_idade)) %>%
tab_cols(VD4002) %>%
tab_weight(weight = V1028) %>%
tab_stat_cases(total_statistic = "w_cases",
total_label = "Total") %>%
tab_pivot()

participacao_idade

|                                 |                        |           VD4002 |                     |
 |                                 |                        | Pessoas ocupadas | Pessoas desocupadas |
 | ------------------------------- | ---------------------- | ---------------- | ------------------- |
 | interaction(V2007, faixa_idade) |     Homem.14 a 17 anos |          24253.0 |              6240.6 |
 |                                 |    Mulher.14 a 17 anos |           9050.8 |              2740.6 |
 |                                 |     Homem.18 a 24 anos |         172483.9 |             42896.0 |
 |                                 |    Mulher.18 a 24 anos |          96130.3 |             43396.9 |
 |                                 |     Homem.25 a 39 anos |         396154.7 |             40011.1 |
 |                                 |    Mulher.25 a 39 anos |         269810.3 |             69330.9 |
 |                                 |     Homem.40 a 59 anos |         367895.6 |             24291.4 |
 |                                 |    Mulher.40 a 59 anos |         262049.8 |             33198.2 |
 |                                 |  Homem.60 anos ou mais |          68748.5 |              1738.6 |
 |                                 | Mulher.60 anos ou mais |          34753.3 |               485.0 |
 |                                 |                 #Total |        1701330.1 |            264329.2 |

In [ ]:
# Exportando Resultado
write.xlsx(participacao_idade, 'p_idade_2021.xlsx')

#Participação no mercado de trabalho por gênero, segundo níveis de instrução

In [ ]:
# Mercado de trabalho por escolaridade
participacao_escolaridade <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
tab_cells(interaction(V2007, VD3004)) %>%
tab_cols(VD4002) %>%
tab_weight(weight = V1028) %>%
tab_stat_cases(total_statistic = "w_cases",
total_label = "Total") %>%
tab_pivot()

participacao_escolaridade

|                            |                                                 |           VD4002 |                     |
 |                            |                                                 | Pessoas ocupadas | Pessoas desocupadas |
 | -------------------------- | ----------------------------------------------- | ---------------- | ------------------- |
 | interaction(V2007, VD3004) |  Homem.Sem instrução e menos de 1 ano de estudo |          38932.0 |              1091.4 |
 |                            | Mulher.Sem instrução e menos de 1 ano de estudo |          13643.5 |                     |
 |                            |     Homem.Fundamental incompleto ou equivalente |         242250.8 |             16260.0 |
 |                            |    Mulher.Fundamental incompleto ou equivalente |         109601.3 |             17120.3 |
 |                            |       Homem.Fundamental completo ou equivalente |          72987.7 |             12105.0 |
 |                            |      Mulher.Fundamental completo ou equivalente |          31213.1 |              9399.1 |
 |                            |           Homem.Médio incompleto ou equivalente |          76930.7 |             15520.3 |
 |                            |          Mulher.Médio incompleto ou equivalente |          37975.3 |             14217.9 |
 |                            |             Homem.Médio completo ou equivalente |         406796.6 |             53109.5 |
 |                            |            Mulher.Médio completo ou equivalente |         270898.1 |             78160.5 |
 |                            |        Homem.Superior incompleto ou equivalente |          49838.0 |              5920.8 |
 |                            |       Mulher.Superior incompleto ou equivalente |          51069.2 |              8978.4 |
 |                            |                         Homem.Superior completo |         141799.9 |             11170.5 |
 |                            |                        Mulher.Superior completo |         157394.0 |             21275.4 |
 |                            |                                          #Total |        1701330.1 |            264329.2 |

In [ ]:
# Exportando Resultado
write.xlsx(participacao_escolaridade, 'p_escolaridade_2021.xlsx')

# Pessoas de 14 anos ou mais por grupamentos ocupacionais

In [ ]:
# Atividades desempenhadas por gênero
grupo_ocupacao <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
tab_cells(interaction(V2007,VD4011)) %>%
tab_cols(VD4001, VD4002) %>%
tab_weight(weight = V1028) %>%
tab_stat_cases(total_statistic = "w_cases",
total_label = "Total") %>%
tab_pivot()

grupo_ocupacao

|                            |                                                                                                             |                       VD4001 |                                   |           VD4002 |                     |
 |                            |                                                                                                             | Pessoas na força de trabalho | Pessoas fora da força de trabalho | Pessoas ocupadas | Pessoas desocupadas |
 | -------------------------- | ----------------------------------------------------------------------------------------------------------- | ---------------------------- | --------------------------------- | ---------------- | ------------------- |
 | interaction(V2007, VD4011) |                                                                                  Homem.Diretores e gerentes |                      23835.8 |                                   |          23835.8 |                     |
 |                            |                                                                                 Mulher.Diretores e gerentes |                      13508.7 |                                   |          13508.7 |                     |
 |                            |                                                             Homem.Profissionais das ciências e intelectuais |                      63936.9 |                                   |          63936.9 |                     |
 |                            |                                                            Mulher.Profissionais das ciências e intelectuais |                      92948.5 |                                   |          92948.5 |                     |
 |                            |                                                               Homem.Técnicos e profissionais de nível médio |                      77406.5 |                                   |          77406.5 |                     |
 |                            |                                                              Mulher.Técnicos e profissionais de nível médio |                      47510.0 |                                   |          47510.0 |                     |
 |                            |                                                                 Homem.Trabalhadores de apoio administrativo |                      45498.0 |                                   |          45498.0 |                     |
 |                            |                                                                Mulher.Trabalhadores de apoio administrativo |                      54227.2 |                                   |          54227.2 |                     |
 |                            |                                       Homem.Trabalhadores dos serviços, vendedores dos comércios e mercados |                     177071.6 |                                   |         177071.6 |                     |
 |                            |                                      Mulher.Trabalhadores dos serviços, vendedores dos comércios e mercados |                     223532.2 |                                   |         223532.2 |                     |
 |                            |                            Homem.Trabalhadores qualificados da agropecuária, florestais, da caça e da pesca |                     195619.6 |                                   |         195619.6 |                     |
 |                            |                           Mulher.Trabalhadores qualificados da agropecuária, florestais, da caça e da pesca |                      71574.0 |                                   |          71574.0 |                     |
 |                            |  Homem.Trabalhadores qualificados, operários e artesões da construção, das artes mecânicas e outros ofícios |                     155596.1 |                                   |         155596.1 |                     |
 

In [ ]:
# Exportando Resultado
write.xlsx(grupo_ocupacao, 'g_ocupacao_2021.xlsx')

# Pessoas de 14 anos ou mais por posição na ocupação

In [ ]:
# Posição na ocupação
posicao_ocupacao <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
tab_cells(interaction(V2007,VD4009)) %>%
tab_cols(VD4001, VD4002) %>%
tab_weight(weight = V1028) %>%
tab_stat_cases(total_statistic = "w_cases",
total_label = "Total") %>%
tab_pivot()

posicao_ocupacao

|                            |                                                                     |                       VD4001 |                                   |           VD4002 |                     |
 |                            |                                                                     | Pessoas na força de trabalho | Pessoas fora da força de trabalho | Pessoas ocupadas | Pessoas desocupadas |
 | -------------------------- | ------------------------------------------------------------------- | ---------------------------- | --------------------------------- | ---------------- | ------------------- |
 | interaction(V2007, VD4009) |  Homem.Empregado no setor privado com carteira de trabalho assinada |                     231664.1 |                                   |         231664.1 |                     |
 |                            | Mulher.Empregado no setor privado com carteira de trabalho assinada |                     130019.3 |                                   |         130019.3 |                     |
 |                            |  Homem.Empregado no setor privado sem carteira de trabalho assinada |                     135367.2 |                                   |         135367.2 |                     |
 |                            | Mulher.Empregado no setor privado sem carteira de trabalho assinada |                      54059.8 |                                   |          54059.8 |                     |
 |                            |       Homem.Trabalhador doméstico com carteira de trabalho assinada |                       2737.5 |                                   |           2737.5 |                     |
 |                            |      Mulher.Trabalhador doméstico com carteira de trabalho assinada |                       8680.0 |                                   |           8680.0 |                     |
 |                            |       Homem.Trabalhador doméstico sem carteira de trabalho assinada |                       7919.6 |                                   |           7919.6 |                     |
 |                            |      Mulher.Trabalhador doméstico sem carteira de trabalho assinada |                      67142.8 |                                   |          67142.8 |                     |
 |                            |  Homem.Empregado no setor público com carteira de trabalho assinada |                       6726.1 |                                   |           6726.1 |                     |
 |                            | Mulher.Empregado no setor público com carteira de trabalho assinada |                       4274.2 |                                   |           4274.2 |                     |
 |                            |  Homem.Empregado no setor público sem carteira de trabalho assinada |                      36805.2 |                                   |          36805.2 |                     |
 |                            | Mulher.Empregado no setor público sem carteira de trabalho assinada |                      41122.1 |                                   |          41122.1 |                     |
 |                            |                                Homem.Militar e servidor estatutário |                      81371.7 |                                   |          81371.7 |                     |
 |                            |                               Mulher.Militar e servidor estatutário |                      75786.9 |                                   |          75786.9 |                     |
 |                            |                                                    Homem.Empregador |                      26548.3 |                                   |          26548.3 |                     |
 |                            |                                                   Mulher.Empregador |                      10933.3 |                                   |          10933.3 |                     |
 |         

In [ ]:
# Exportando Resultado
write.xlsx(posicao_ocupacao, 'p_ocupacao_2021.xlsx')

# Rendimento médio mensal efetivamente recebido por gênero, segundo grupamentos ocupacionais

In [ ]:
# Rendimento médio
rendimento <- pnadc_2021 %>% filter(UF == "Amazonas") %>%
tab_cells(VD4020) %>%
tab_cols(total(), V2007) %>%
tab_rows(VD4011, total()) %>%
tab_weight(weight = V1028) %>%
tab_stat_fun(Média = w_mean) %>%
tab_pivot()

rendimento

Warning message in as.data.frame.numeric(x[[i]], optional = TRUE, nm = "x[[i]]", :
“Direct call of 'as.data.frame.numeric()' is deprecated.  Use 'as.data.frame.vector()' or 'as.data.frame()' instead”


|        |                                                                                                      |        |       | #Total |  V2007 |        |
 |        |                                                                                                      |        |       |        |  Homem | Mulher |
 | ------ | ---------------------------------------------------------------------------------------------------- | ------ | ----- | ------ | ------ | ------ |
 | VD4011 |                                                                                 Diretores e gerentes | VD4020 | Média | 4968.0 | 5492.1 | 4032.3 |
 |        |                                                            Profissionais das ciências e intelectuais | VD4020 | Média | 3842.7 | 4487.2 | 3395.1 |
 |        |                                                              Técnicos e profissionais de nível médio | VD4020 | Média | 2772.2 | 3123.6 | 2186.2 |
 |        |                                                                Trabalhadores de apoio administrativo | VD4020 | Média | 1880.3 | 2111.9 | 1681.5 |
 |        |                                      Trabalhadores dos serviços, vendedores dos comércios e mercados | VD4020 | Média | 1317.9 | 1631.6 | 1061.5 |
 |        |                           Trabalhadores qualificados da agropecuária, florestais, da caça e da pesca | VD4020 | Média |  628.0 |  625.6 |  646.3 |
 |        | Trabalhadores qualificados, operários e artesões da construção, das artes mecânicas e outros ofícios | VD4020 | Média | 1303.7 | 1355.7 | 1051.5 |
 |        |                                                    Operadores de instalações e máquinas e montadores | VD4020 | Média | 1471.0 | 1488.7 | 1311.5 |
 |        |                                                                                Ocupações elementares | VD4020 | Média |  897.3 |  978.0 |  798.8 |
 |        |                                          Membros das forças armadas, policiais e bombeiros militares | VD4020 | Média | 4472.0 | 4369.4 | 5204.9 |
 |        |                                                                               Ocupações maldefinidas | VD4020 | Média |        |        |        |
 | #Total |                                                                                               VD4020 |  Média |       | 1722.9 | 1796.0 | 1602.9 |

In [ ]:
# Exportando Resultado
write.xlsx(rendimento, 'rendaocup_2021.xlsx')